# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# This should create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Setting connection to the keyspace created above
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# ---- ANSWERS ----

### Creating table to answer question #1
#### Considering that the query we need to create demands only the information that was heard during session Id 338 and itemInSession 4, disregarding the user who has listened to the song, it was necessary only to include these two columns to the primary key.

In [10]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session"
query = query +   "(session_id int, item_in_session int, artist text, song_title text, song_length double, PRIMARY KEY (session_id, item_in_session))"
                       
try:
    session.execute(query)
except Exception as e:
    print(e)                   

### Inserting data to answer question #1

In [11]:
# Opening and reading the event_datafile_new.csv file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Inserting the data to the table that was created above (songs_by_session).
        query = "INSERT INTO songs_by_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
# Assigning the column element from csv file that will be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Answer to question #1
#### The following code executes a SELECT statement on the table songs_by_session and prints the artist, song title and song length, that was heard during the session 330, and itemInSession is equal to 4, using a Pandas DataFrame.

In [12]:
query = "SELECT artist, song_title, song_length FROM songs_by_session WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
print(df)

      artist                       song_title  song_length
0  Faithless  Music Matters (Mark Knight Dub)     495.3073


### Creating table to answer question #2
#### Considering that the query we need to create demands only the information of the userid 10, during the session id 182, it was necessary only to include these two columns to the primary key, besides the insertion of the item in session as a Clustering Key, to sort the data as requested by the question.

In [13]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user"
query = query +   "(session_id int, user_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY ((session_id,user_id),item_in_session))"
                       
try:
    session.execute(query)
except Exception as e:
    print(e)

### Inserting data to answer question #2

In [14]:
# Opening and reading the event_datafile_new.csv file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Inserting the data to the table that was created above (songs_by_user).
        query = "INSERT INTO songs_by_user (session_id, user_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
# Assigning the column element from csv file that will be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

### Answer to question #2
#### The following code executes a SELECT statement on the table songs_by_user and prints the artist, song title, first and last name of the user, for the user id 10 and session id 182, using a Pandas DataFrame.

In [15]:

query = "SELECT artist, song_title, first_name, last_name FROM songs_by_user WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
print(df)

              artist                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


### Creating table to answer question #3
#### Considering that the query we need to create demands to show every user name who listened to a specific song, and no mather how many times, or even in which session it was heard, it was necessary to add only the fields song title and user id to the primary key.
#### The field user id was necessary, because it's possible to have more than on user with the same first and last name.

In [16]:
query = "CREATE TABLE IF NOT EXISTS users_listening"
query = query +   "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"
                       
try:
    session.execute(query)
except Exception as e:
    print(e)

### Inserting data to answer question #3

In [17]:
# Opening and reading the event_datafile_new.csv file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Inserting the data to the table that was created above (users_listening).
        query = "INSERT INTO users_listening (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s,%s,%s,%s)"
# Assigning the column element from csv file that will be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### Answer to question #3
#### The following code executes a SELECT statement on the table users_lintening and prints users first and last name for those who has listened to the song 'All Hands Against His Own'.

In [18]:
query = "SELECT first_name, last_name FROM users_listening WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

#### The following code drops all the tables.

In [19]:
query = "DROP TABLE IF EXISTS songs_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS songs_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS users_listening"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()